In [ ]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from IPython.display import display

In [ ]:
plt.rcParams['axes.xmargin'] = 0
plt.rcParams['axes.ymargin'] = 0

In [ ]:
def periodically_continued(a, b):
    interval = b - a
    return lambda f: lambda x: f((x - a) % interval + a)

In [ ]:
C = sp.symbols('C', real=True, positive=True)
alphap = sp.symbols(r'\alpha_{p}')
dx = sp.symbols(r'\Delta{x}', real=True, positive=True)
p = sp.symbols('p', real=True)
m = sp.symbols('m', real=True)

In [ ]:
def expon(space, time):
    return alphap ** time * sp.exp(sp.I * 2 * sp.pi * p * (m + space) * dx)

In [ ]:
def getErrors(eqn, name, *, space_step=1/300, time_step=1/500, velocity=1, save=True, disp=True):
    solns = sp.solve(eqn, alphap)
    
    soln = solns[-1].simplify()
    if disp:
        display(soln)
    
    betap = sp.Abs(soln).simplify()
    if disp:
        display(betap)
    
    epsilonp = sp.arg(soln).simplify()
    if disp:
        display(epsilonp)
    
    ps = np.linspace(0, 1/space_step, 1000)
    courant = velocity * time_step / space_step
    
    betap_ps = ps.copy()
    betap_func = np.vectorize(sp.lambdify(p, betap.subs(C, courant).subs(dx, space_step)))
    betap_vals = np.real(betap_func(betap_ps))
        
    fig, betap_ax = plt.subplots()
    betap_ax.plot(betap_ps, betap_vals, label="Actual value")
    betap_ax.set_xlabel(r"$p$")
    betap_ax.set_ylabel(r"$\beta_p$")
    betap_ax.set_ylim(0, 1.5)
#     betap_ax.legend()
    
    if save:
        plt.savefig(f'figures/linear/{name.lower()}/{name.lower()}-beta.pdf')
    
    plt.show()
    
    epsilonp_ps = ps.copy()
    epsilonp_func = sp.lambdify(p, epsilonp.subs(C, courant).subs(dx, space_step))
    epsilonp_vals = np.real(epsilonp_func(epsilonp_ps))
    
    epsilonp_poss = np.where(np.abs(np.diff(epsilonp_vals)) >= 0.5)
    for pos in epsilonp_poss:
        epsilonp_ps[pos] = np.nan
        epsilonp_vals[pos] = np.nan
    
    theta_ps = ps.copy()
    calc_theta_vals = lambda p : - 2 * np.pi * p * velocity * time_step
    calc_theta_vals = periodically_continued(-1 / (2 * space_step), 1 / (2 * space_step))(calc_theta_vals)
    theta_vals = calc_theta_vals(theta_ps)
    
    thetas_poss = np.where(np.abs(np.diff(theta_vals)) >= 0.5)
    for pos in thetas_poss:
        theta_ps[pos] = np.nan
        theta_vals[pos] = np.nan
    
    fig, epsilonp_ax = plt.subplots()
    epsilonp_ax.plot(epsilonp_ps, epsilonp_vals, label="Actual value", zorder=5)
    epsilonp_ax.plot(theta_ps, theta_vals, label="Desired value", zorder=0)
    epsilonp_ax.set_xlabel(r"$p$")
    epsilonp_ax.set_ylabel(r"$\epsilon_p$")
    epsilonp_ax.set_ylim(-3, 3)
    epsilonp_ax.legend(loc="lower right", fontsize=13)
    
    if save:
        plt.savefig(f'figures/linear/{name.lower()}/{name.lower()}-epsilon.pdf')
    
    plt.show()

In [ ]:
eqn = sp.Eq(expon(0, 1), expon(0, 0) - C * (expon(0, 0) - expon(-1, 0)))
getErrors(eqn, 'Upwind-Forward', save=True)

In [ ]:
eqn = sp.Eq((1 + C) * expon(0, 1) - C * expon(-1, 1), expon(0, 0))
getErrors(eqn, 'Upwind-Backward', save=True)

In [ ]:
eqn = sp.Eq((1 + C / 2) * expon(0, 1) - C / 2 * expon(-1, 1), (1 - C / 2) * expon(0, 0) + C / 2 * expon(-1, 0))
getErrors(eqn, 'Upwind-Trapezoidal', save=True)

In [ ]:
eqn = sp.Eq(expon(0, 1), expon(0, 0) - C / 2 * expon(1, 0) + C / 2 * expon(-1, 0))
getErrors(eqn, 'Centered-Forward', save=True)

In [ ]:
eqn = sp.Eq(C / 2 * expon(1, 1) +  expon(0, 1) - C / 2 * expon(-1, 1), expon(0, 0))
getErrors(eqn, 'Centered-Backward', save=True)

In [ ]:
eqn = sp.Eq(expon(0, 1) + C / 4 * (expon(1, 1) - expon(-1, 1)), expon(0, 0) - C / 4 *(expon(1, 0) - expon(-1, 0)))
getErrors(eqn, 'Centered-Trapezoidal', save=True)

In [ ]:
eqn = sp.Eq(expon(0, 1), expon(0, -1) - C * (expon(1, 0) - expon(-1, 0)))
getErrors(eqn, 'Leapfrog', save=True)

In [ ]:
eqn = sp.Eq(expon(0, 1), expon(0, 0) - C / 2 * (expon(1, 0) - expon(-1, 0)) + C ** 2 / 2 * (expon(1, 0) + expon(-1, 0) - 2 * expon(0, 0)))
getErrors(eqn, 'Lax-Wendroff', save=True)